as reference we will follow this tutorial [**Model deployment using AWS SageMaker**](https://medium.com/datadriveninvestor/model-deployment-using-aws-sagemaker-8116adea7184)

first of all we will have to train a custom tf 2.x model and generate .h5 or .hdf5 format's weight file that we can deploy using aws sagemaker, my training kernel is private but i trained a model on  [Stanford Sentiment Treebank v2 (SST2)](https://www.kaggle.com/atulanandjha/stanford-sentiment-treebank-v2-sst2)  dataset using this baseline notebook [Understanding cross lingual models](https://www.kaggle.com/mobassir/understanding-cross-lingual-models) i uploaded my models weight file in aws s3 bucket, opened new aws notebook instance and then,

upgrading pip

In [ ]:
!python -m pip install --upgrade pip


installing tf 2.2.0 that i used during training my model

In [ ]:
!pip3 install tensorflow==2.2.0

In [ ]:
!pip install tensorflow-gpu==2.2.0

installing transformers 2.11.0 that i used during model training

In [ ]:
!pip3 install transformers==2.11.0

In [ ]:
!pip install --upgrade sagemaker

# Cool Imports

In [ ]:
%matplotlib inline
import boto3, re, sys, math, json, os, sagemaker, urllib.request
from sagemaker import get_execution_role
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sagemaker.tensorflow.serving import Model
from sagemaker.predictor import csv_serializer 
from sagemaker.utils import name_from_base

import tensorflow as tf
from tensorflow.keras.layers import Dense, Input,  Embedding
import transformers
from transformers import AutoTokenizer

sess = sagemaker.Session()

role = get_execution_role()
print(role) # This is the role that SageMaker would use to leverage AWS resources (S3, CloudWatch) on your behalf

prefix = 'xlmrlarge/' #Replace with the prefix under which you want to store the data if needed

arn:aws:iam::006159542779:role/service-role/AmazonSageMaker-ExecutionRole-20201216T154428


In [ ]:
tf.__version__

'2.2.0'

In [ ]:
transformers.__version__

'2.11.0'

in your working directory you will have to create 2 folders called "model" and  "code" as shown here : https://github.com/mobassir94/Deploy-trained-TensorFlow-2.x-models-using-Amazon-SageMaker/blob/main/jupyter.PNG

![](https://github.com/mobassir94/Deploy-trained-TensorFlow-2.x-models-using-Amazon-SageMaker/blob/main/jupyter.PNG)


the model folder structure should be exactly like this : https://github.com/mobassir94/Deploy-trained-TensorFlow-2.x-models-using-Amazon-SageMaker/blob/main/folderstructure.PNG

![](https://github.com/mobassir94/Deploy-trained-TensorFlow-2.x-models-using-Amazon-SageMaker/blob/main/folderstructure.PNG)

my folder structure looks like this : https://github.com/mobassir94/Deploy-trained-TensorFlow-2.x-models-using-Amazon-SageMaker/blob/main/myfolderstructure.PNG

![](https://github.com/mobassir94/Deploy-trained-TensorFlow-2.x-models-using-Amazon-SageMaker/blob/main/myfolderstructure.PNG)

yours one should look similar (once you run similar code like the code below that fits your case)

be careful, if the folder structure is not similar(like i showed above) then sagemaker won't find your model for making inference,,it costs me few hours

my weight file was saved in .h5 format. my file name was "model_checkpoint_0.h5" and i uploaded that in s3 bucket, the code below downloads that weight file from s3 bucket and stores that file in my working directory,then i am loading my model and weight file into that model then finally after loading my weight file into my model i am converting it into tf servable format using this line of code : tf.saved_model.save(model, save_path)

this is very important,
see my save path is : save_path = os.path.join("/home/ec2-user/SageMaker/model/1/")

you can see inside model folder i have a folder called "1" which is basically my model's version and inside that "1" folder i am converting my weight file into tf servable format,after running the code below,my model folder looks like this : https://github.com/mobassir94/Deploy-trained-TensorFlow-2.x-models-using-Amazon-SageMaker/blob/main/myfolderstructure.PNG

sagemaker expects model folder structure like that,if you make mistake then sagemaker won't find your model at all

In [1]:
''' 
modelpath = './model_checkpoint_0.h5'


def regular_encode(texts, tokenizer, maxlen=512):
    enc_di = tokenizer.batch_encode_plus(
        texts, 
        return_attention_masks=False, 
        return_token_type_ids=False,
        pad_to_max_length=True,
        max_length=maxlen
    )
    
    return np.array(enc_di['input_ids'])
    

#from keras_radam import RAdam
def build_model(transformer, loss='binary_crossentropy', max_len=512):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    sequence_output = transformer(input_word_ids)[0]
    cls_token = sequence_output[:, 0, :]
    #x = tf.keras.layers.Dropout(0.3)(cls_token)
    out = Dense(1, activation='sigmoid')(cls_token)
    
    
    model = Model(inputs=input_word_ids, outputs=out)
    #model.compile(RAdam( warmup_proportion=0.1, min_lr=1e-5), loss=loss, metrics=[tf.keras.metrics.AUC()])
    
    return model
    
    
strategy = tf.distribute.MirroredStrategy()


from tensorflow.keras.callbacks import Callback
from sklearn.metrics import accuracy_score, roc_auc_score
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, CSVLogger

from tensorflow.keras.models import Model

from tensorflow.keras.optimizers import Adam
#from tokenizers import BertWordPieceTokenizer
from tensorflow.keras.layers import Dense, Input, Dropout, Embedding
from tensorflow.keras.layers import LSTM, GRU, Conv1D, SpatialDropout1D

from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import activations
from tensorflow.keras import constraints
from tensorflow.keras import initializers
from tensorflow.keras import regularizers

import tensorflow.keras.backend as K
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.activations import *
from tensorflow.keras.constraints import *
from tensorflow.keras.initializers import *
from tensorflow.keras.regularizers import *


MODEL = 'jplu/tf-xlm-roberta-large'
tokenizer = AutoTokenizer.from_pretrained(MODEL)
maxlen = 192
with strategy.scope():
    transformer_layer = transformers.TFXLMRobertaModel.from_pretrained(MODEL)
    model = build_model(transformer_layer, max_len=maxlen)
model.summary()

#https://stackoverflow.com/questions/61421528/read-h5-file-using-aws-boto3
#https://stackoverflow.com/questions/29378763/how-to-save-s3-object-to-a-file-using-boto3

import boto3
import h5py
s3_client = boto3.client('s3')

s3_client.download_file('xlmroberta', 'model_checkpoint_0.h5', 'model_checkpoint_large.h5')

#os.listdir('/home/ec2-user/SageMaker/')

path = '/home/ec2-user/SageMaker/model_checkpoint_large.h5'
model.load_weights(path)


save_path = os.path.join("/home/ec2-user/SageMaker/model/1/")
tf.saved_model.save(model, save_path)

'''

' \nmodelpath = \'./model_checkpoint_0.h5\'\n\n\ndef regular_encode(texts, tokenizer, maxlen=512):\n    enc_di = tokenizer.batch_encode_plus(\n        texts, \n        return_attention_masks=False, \n        return_token_type_ids=False,\n        pad_to_max_length=True,\n        max_length=maxlen\n    )\n    \n    return np.array(enc_di[\'input_ids\'])\n    \n\n#from keras_radam import RAdam\ndef build_model(transformer, loss=\'binary_crossentropy\', max_len=512):\n    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")\n    sequence_output = transformer(input_word_ids)[0]\n    cls_token = sequence_output[:, 0, :]\n    #x = tf.keras.layers.Dropout(0.3)(cls_token)\n    out = Dense(1, activation=\'sigmoid\')(cls_token)\n    \n    \n    model = Model(inputs=input_word_ids, outputs=out)\n    #model.compile(RAdam( warmup_proportion=0.1, min_lr=1e-5), loss=loss, metrics=[tf.keras.metrics.AUC()])\n    \n    return model\n    \n    \nstrategy = tf.distribute.Mirrored

**we run the cell above for once,then we can comment out the cell because we have already converted our weight file into tf servable format and we don't have to do it over and over again,now inside our model folder we have converted weight file in tf servable format and now we don't need the cell above anymore, sagemaker will use our new converted tf servable format's model for making inference,now we can also delete our .h5 weight file from s3 bucket or from our working environment because we don't need that anymore**

In [2]:
!pwd

/content


In [ ]:
!ls /home/ec2-user/SageMaker

basemodelDeployXLMrXLARGE.ipynb  code  lost+found  model


In [ ]:
os.listdir('/home/ec2-user/SageMaker/model')

['xlmrlargewithcode.tar.gz', '1', '.ipynb_checkpoints']

we will do cd to our model directory as we need to make tar of that folder and then will upload the tar.gz archive file containing Saved Model to S3 bucket.



In [ ]:
cd /home/ec2-user/SageMaker/model/

In [ ]:
ls

basemodelDeployXLMrXLARGE.ipynb  code/  lost+found/  model/


In [ ]:
!pwd

/home/ec2-user/SageMaker


In [ ]:
os.listdir('/home/ec2-user/SageMaker/model/1')

['assets', 'saved_model.pb', '.ipynb_checkpoints', 'variables']

i repeat, we need to make sure that after making tar our tar file contains folder structure like this : https://github.com/mobassir94/Deploy-trained-TensorFlow-2.x-models-using-Amazon-SageMaker/blob/main/myfolderstructure.PNG

for example this tar command is wrong : !tar -C "$PWD" -czf xlmrbasemodel.tar.gz --absolute-names /home/ec2-user/SageMaker/model/

it will break folder structure, the right command is : 
!tar -czvf xlmrlargewithcode.tar.gz 1/

In [ ]:
%%time
#export_dir = '/home/ec2-user/SageMaker/model/1' 

#!tar -czvf xlmrbasemodel.tar.gz --absolute-names /home/ec2-user/SageMaker

#!tar -C "$PWD" -czf xlmrmodel.tar.gz --absolute-names /home/ec2-user/SageMaker/model/

#!tar -C "$PWD" -czf xlmrbasemodel.tar.gz --absolute-names /home/ec2-user/SageMaker/model/

!tar -czvf xlmrlarge.tar.gz 1/

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 7.63 µs


In [ ]:
os.listdir('/home/ec2-user/SageMaker/model')

['xlmrlargewithcode.tar.gz', '1', '.ipynb_checkpoints']

In [ ]:
#!unzip /home/ec2-user/SageMaker/xlmrmodel.tar.gz

#!tar -xf /home/ec2-user/SageMaker/xlmrmodel.tar.gz -C --absolute-names /home/ec2-user/SageMaker/

model_data holds the s3 location of our tar.gz model file

In [ ]:
bucket_name = 'xlmroberta'
model_data =  "s3://xlmroberta/xlmrlarge/model/xlmrlarge.tar.gz"

#model_data = sess.upload_data(path='xlmrlarge.tar.gz', bucket=bucket_name, key_prefix = prefix+"model") #, key_prefix = prefix+"model"
model_data

's3://xlmroberta/xlmrlarge/model/xlmrlarge.tar.gz'

In [ ]:
cd /home/ec2-user/SageMaker/

inside code directory we have 2 things: 
1. inference.py that contains your input,output handler and other things like tokenizer that can be used for making inference using your custom model,my inference.py looks like this(inside code folder) : https://github.com/mobassir94/Deploy-trained-TensorFlow-2.x-models-using-Amazon-SageMaker/blob/main/inference.py
2. requirements.txt contains all the packages that needs to be installed for making inference using our model

my code folder looks like this : https://github.com/mobassir94/Deploy-trained-TensorFlow-2.x-models-using-Amazon-SageMaker/blob/main/codefolder.PNG

In [ ]:
'''
we are telling sagemaker to use inference.py file from code directory,it will automatically install requirements.txt file and also we are 
providing it the model xlmrlarge.tar.gz file that can be found inside our s3 bucket and in this location "s3://xlmroberta/xlmrlarge/model/xlmrlarge.tar.gz"
yours location can be different
'''
model = Model(entry_point="inference.py",
              source_dir="code",
              model_data=model_data,
              name=name_from_base("xlmrlarge"),
              role=role,
              framework_version = '2.2.0'
             )
''' 
model = Model(
              model_data=model_data,
              name=name_from_base("xlmrlarge"),
              role=role,
              framework_version = '2.2.0'
             )


'''

The class sagemaker.tensorflow.serving.Model has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


' \nmodel = Model(\n              model_data=model_data,\n              name=name_from_base("xlmrlarge"),\n              role=role,\n              framework_version = \'2.2.0\'\n             )\n\n\n'

the cell below will deploy our model using sagemaker and create endpoint,note that instance_type='ml.t2.xlarge' is the type of my notebook instance that i created from aws notebook instance section,yours one can be different here

In [ ]:
%%time
predictor = model.deploy(initial_instance_count=1,   instance_type='ml.t2.xlarge')

update_endpoint is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


---------------!CPU times: user 8min 39s, sys: 14.2 s, total: 8min 53s
Wall time: 16min 3s


In [ ]:
inputs = "ঢাকা বিমানবন্দরে ২৫০ কেজি'র বোমা: এটি কীভাবে এলো, বিস্ফোরিত হয়নি কেন"
#inputs2 = test.Words[:5].tolist()
ck=json.dumps(inputs)
ck

'"\\u09a2\\u09be\\u0995\\u09be \\u09ac\\u09bf\\u09ae\\u09be\\u09a8\\u09ac\\u09a8\\u09cd\\u09a6\\u09b0\\u09c7 \\u09e8\\u09eb\\u09e6 \\u0995\\u09c7\\u099c\\u09bf\'\\u09b0 \\u09ac\\u09cb\\u09ae\\u09be: \\u098f\\u099f\\u09bf \\u0995\\u09c0\\u09ad\\u09be\\u09ac\\u09c7 \\u098f\\u09b2\\u09cb, \\u09ac\\u09bf\\u09b8\\u09cd\\u09ab\\u09cb\\u09b0\\u09bf\\u09a4 \\u09b9\\u09af\\u09bc\\u09a8\\u09bf \\u0995\\u09c7\\u09a8"'

for sanity check,the code cell below sends our input string to our deploymed model and from there using our inference.py script it returning us a predicted answer that gets stored in response variable

In [ ]:
response = predictor.predict(json.dumps(inputs))
#response = predictor.predict(x_test1)

In [ ]:
print(response)

{'predictions': 'Negative'}


In [ ]:
response['predictions']

'Negative'

In [ ]:
predictor.endpoint

The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


'xlmrlarge-2020-12-20-14-17-25-925'

That's it, we have deployed our model,created endpoint,made inference using single sentence and got predicted answer as well

In [ ]:
endpoint_name = predictor.endpoint

endpoint_name

The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


'xlmrlarge-2020-12-20-14-17-25-925'

now we will see how we can invoke sagemaker endpoint from lambda or Integrate Lambda with Sagemaker, you can follow the instructions of this tutorial : https://www.youtube.com/watch?v=-iU36P8hizs

you need to design lambda_handler according to the updated "inference.py" file or updated input,output handler that you gonna use to solve your data problem

my lambda function looks like this :

In [ ]:

import os
import io
import boto3
import json
import csv

# grab environment variables
ENDPOINT_NAME = 'your endpoint name'

runtime= boto3.client('runtime.sagemaker', 
                     region_name='us-east-1', # make sure to set correct region
                     aws_access_key_id='your aws access key', 
                      # These you get from AWS, for your account
                      aws_secret_access_key='your aws secret key')

def lambda_handler(event, context):
    print("Received event: " + json.dumps(event, indent=2))
    
    data = json.loads(json.dumps(event))
    payload = data['data']
    print(payload)
    
    response = runtime.invoke_endpoint(EndpointName=ENDPOINT_NAME,
                                       ContentType='application/json',
                                       Body=payload)
    print(response)
    result = json.loads(response['Body'].read().decode())
    print(result)
    predicted_label = result['predictions']
    
    return predicted_label

so once,we have this lambda,to really expose it,we need to put an API gateway infront of it,you can create new api by following this tutorial : https://www.youtube.com/watch?v=-iU36P8hizs

once done,you will get invoke URL that you can use from postman to make post request. you can see here : https://github.com/mobassir94/Deploy-trained-TensorFlow-2.x-models-using-Amazon-SageMaker/blob/main/postmandemo.PNG that for the romanic bangla sentence "ansary vhai khub valo manush" in english which means "ansary bhai is a very good man" we get positive sentiment as a result which is appropriate for the sample we used for making prediction using our model